In [177]:
# We are going to predict life which is continous value. So this Regression problem

import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [178]:
df = pd.read_csv('gapminder.csv')

In [179]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139 entries, 0 to 138
Data columns (total 10 columns):
population         139 non-null int64
fertility          139 non-null float64
HIV                139 non-null float64
CO2                139 non-null float64
BMI_male           139 non-null float64
GDP                139 non-null int64
BMI_female         139 non-null float64
life               139 non-null float64
child_mortality    139 non-null float64
Region             139 non-null object
dtypes: float64(7), int64(2), object(1)
memory usage: 10.9+ KB


In [180]:
df.head()

,population,fertility,HIV,CO2,BMI_male,GDP,BMI_female,life,child_mortality,Region
0,34811059,2.73,0.1,3.328945,24.59620,12314,129.9049,75.3,29.5,Middle East & North Africa
1,19842251,6.43,2.0,1.474353,22.25083,7103,130.1247,58.3,192.0,Sub-Saharan Africa
2,40381860,2.24,0.5,4.785170,27.50170,14646,118.8915,75.5,15.4,America
3,2975029,1.40,0.1,1.804106,25.35542,7383,132.8108,72.5,20.0,Europe & Central Asia
4,21370348,1.96,0.1,18.016313,27.56373,41312,117.3755,81.5,5.2,East Asia & Pacific


In [181]:
X = df.drop(['life','Region'],axis=1)
y = df.life

In [182]:
X = pd.concat([pd.get_dummies(df['Region']),X],axis=1)
X.head()

,America,East Asia & Pacific,Europe & Central Asia,Middle East & North Africa,South Asia,Sub-Saharan Africa,population,fertility,HIV,CO2,BMI_male,GDP,BMI_female,child_mortality
0,0,0,0,1,0,0,34811059,2.73,0.1,3.328945,24.59620,12314,129.9049,29.5
1,0,0,0,0,0,1,19842251,6.43,2.0,1.474353,22.25083,7103,130.1247,192.0
2,1,0,0,0,0,0,40381860,2.24,0.5,4.785170,27.50170,14646,118.8915,15.4
3,0,0,1,0,0,0,2975029,1.40,0.1,1.804106,25.35542,7383,132.8108,20.0
4,0,1,0,0,0,0,21370348,1.96,0.1,18.016313,27.56373,41312,117.3755,5.2


In [183]:
from sklearn.model_selection import train_test_split as tts , GridSearchCV , RandomizedSearchCV
X_train,X_test,y_train,y_test = tts(X,y,random_state=42,test_size=0.75)

In [184]:
from sklearn.tree import DecisionTreeRegressor
dtr = DecisionTreeRegressor()
dtr

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [185]:
dtr.fit(X_train,y_train)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [186]:
y_pred = dtr.predict(X_test)

In [187]:
r_score = dtr.score(X_test,y_test)
print('r_score : '+ str(r2_score))

r_score : <function r2_score at 0x7f8c54b77bf8>


In [188]:
# Check performance using various metrices
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score

In [189]:
def check_common_metrices_score(test,pred):
    print("mean absolute error is : "+ str(mean_absolute_error(test,pred)))
    print("Mean squared error is : "+ str(mean_squared_error(test,pred)))
    print("r2 score is : "+ str(r2_score(test,pred)))

In [190]:
check_common_metrices_score(y_pred,y_test)

mean absolute error is : 3.8514285714285714
Mean squared error is : 23.686666666666678
r2 score is : 0.738404938422877


In [191]:
features_coeff = dtr.feature_importances_

In [192]:
feature_coeff_map = dict(zip(X.columns,features_coeff))
print(sorted(feature_coeff_map.items(), key=lambda item: item[1],reverse=True))
best_features = [k for k, v in sorted(feature_coeff_map.items(), key=lambda item: item[1],reverse=True)]

[('child_mortality', 0.8711698948897385), ('HIV', 0.04800616497260276), ('BMI_male', 0.027944837131082645), ('population', 0.027524456778849373), ('fertility', 0.02114568471253356), ('GDP', 0.0021514469471961705), ('BMI_female', 0.0016182681251659683), ('America', 0.00031837555721215177), ('Europe & Central Asia', 8.798084871549107e-05), ('CO2', 2.6312029523094356e-05), ('Sub-Saharan Africa', 6.5780073803248884e-06), ('East Asia & Pacific', 0.0), ('Middle East & North Africa', 0.0), ('South Asia', 0.0)]


In [233]:
# parameter tuning  criterion="entropy", max_depth=10, min_samples_split=0.06,class_weight="balanced"
params = {'criterion':['friedman_mse','mse'],'max_depth':np.arange(2,20),'min_samples_split':np.arange(2,6),'min_weight_fraction_leaf':np.arange(0.01,0.10,0.01)}

In [242]:
dtr = DecisionTreeRegressor()

# GridSearchCV with default midel, params, and cv = 10
dtr_cv = GridSearchCV(estimator=dtr,param_grid=params,cv = 10)

In [235]:
dtr_cv.fit(X,y)

GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.05,
           presort=False, random_state=None, splitter='best'),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'criterion': ['friedman_mse', 'mse'], 'max_depth': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19]), 'min_samples_split': array([2, 3, 4, 5]), 'min_weight_fraction_leaf': array([0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [236]:
dtr_cv.best_estimator_

DecisionTreeRegressor(criterion='friedman_mse', max_depth=3,
           max_features=None, max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.04, presort=False, random_state=None,
           splitter='best')

In [237]:
dtr_cv.best_params_

{'criterion': 'friedman_mse',
 'max_depth': 3,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.04}

In [238]:
dtr_cv.best_score_

0.8261369203660636

In [239]:
dtr_best_model = dtr_cv.best_estimator_
dtr_best_model.fit(X_train,y_train)
# check_common_metrices_score(y_test,dtr_cv.predict(y_test))

DecisionTreeRegressor(criterion='friedman_mse', max_depth=3,
           max_features=None, max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.04, presort=False, random_state=None,
           splitter='best')

In [240]:
y_pred = dtr_best_model.predict(X_test)

In [241]:
check_common_metrices_score(y_test,y_pred)

mean absolute error is : 3.2343862433862407
Mean squared error is : 17.065942184009376
r2 score is : 0.7877031863242199


In [232]:
np.arange(0.01,0.10,0.01)

array([0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09])